# Section 3 (Objects and Visualizations)

* [Babynames Dataset](#Babynames-Dataset)
* [Objects And Classes](#Objects-And-Classes)
    * [Constructors](#Constructors)
    * [Attributes](#Attributes)
    * [`.__str__()`](#.__str__())
    * [Custom Methods](#Custom-Methods)
    * [`.__str__()` And `.__repr__()`](#.__str__()-And-.__repr__())
    * [Documentation]()
    * [More Dunder Methods](#More-Dunder-Methods)
    * [The Main Takeaways](#The-Main-Takeaways)
* [Random Numbers](#Random-Numbers)
* [Numpy](#Numpy)
    * [What Is Numpy, And Why Do We Need It?](#What-Is-Numpy,-And-Why-Do-We-Need-It?)
    * [Basic Numpy Usage](#Basic-Numpy-Usage)
    * [Basic Numpy Math](#Basic-Numpy-Math)
    * [`arange()`](#arange())
    * [Various Array Behaviors](#Various-Array-Behaviors)
    * [Multidimensional Arrays](#Multidimensional-Arrays)
    * [Slicing](#Slicing)
        * [An aside about how columns and rows are requested.](#An-aside-about-how-columns-and-rows-are-requested.)
    * [Other Common Numpy Tools](#Other-Common-Numpy-Tools)
    * [Aggregate Functions](#Aggregate-Functions)
* [Matplotlib](#Matplotlib)
    * [Saving The Graph As An Image](#Saving-The-Graph-As-An-Image)
    * [More Graph Types](#More-Graph-Types)
    * [Stupid Matplotlib Tricks](#Stupid-Matplotlib-Tricks)
    * [The Object-Oriented Interface](#The-Object-Oriented-Interface)


# Babynames Dataset

In [ ]:
#########################################################
###    Run This Code To Download And Load The Data    ###
#########################################################

from zipfile import ZipFile
def readnames(path):
    '''
    This function creates a list of baby name data, which
    is the dataset on baby names from the Social Security
    Administration.  Data is grouped by state, sex, year,
    name, and then finally the number of babies born that
    year with that name.  Names that occurred less than 5
    times that year in that state are not included.
    The code also saves a copy of the dataset in a file on
    your computer.  The next time that you run the script,
    it will first try to load the dataset directly from your
    computer.  If it doesn't find it, then it will go ahead
    and try to download the file.
    
    Arguments:
    path - The path to the babynames zip file
    
    Returns:
    A list of tuples containing the data in the form of:
      (state, sex, year, name, count)
    '''

    babyNamesRaw = []
    
    # Open the .zip file.  `babyzip` is the variable that
    # represents the .zip file.
    with ZipFile(path) as babyzip:
        
        # .zip files are a compressed container for
        # potentially many other files.
        # Iterate through the files and process the ones
        # that represent state data.
        for filename in [filename for filename in babyzip.namelist() if ".TXT" in filename]:
            print("Loading data for " + (filename[:2]))
            
            # Open the file from the .zip file.
            # The file is opened in RAM, so no additional
            # disk space is required.
            with babyzip.open(filename) as state:
                
                # Iterate through each line in the file
                # and append to the `babyNamesRaw` list.
                for line in [line for line in state.read().decode().splitlines() if line != '']:
                    
                    # Convert numerical data (year and count).
                    state, sex, year, name, count = line.split(',')
                    babyNamesRaw.append((state, sex, int(year), name, int(count)))

    return babyNamesRaw


zipname = "babynames.zip"
try:
    # Assume the file exists.  Try to open it.
    babyList = readnames(zipname)
except:
    # The file open() failed.
    try:
        # Try to download the file.
        from urllib.request import urlopen
        print("Fetching the file from the interwebs...")
        print("NOTE: This will take a while. The file is around 20 megabytes!  And that's compressed, too!")
        response = urlopen("https://www.ssa.gov/oact/babynames/state/namesbystate.zip")

        # Write the downloaded data to a file.
        with open(zipname, 'wb') as babyfile:
            babyfile.write(response.read())
        babyList = readnames(zipname)
    except:
        # Something else went wrong.
        # Raise another Exception, which will cause
        # the script to stop running (with an error).
        raise Exception("Could not load file.")

print(f'Baby Names data is loaded.  There are {len(babyList)} records.')


# Objects And Classes

We have already seen many types of **objects** in Python.  **Strings**, **Lists**, **Dictionaries**, **Tuples**, **Sets**, and even **Functions** are all types of **objects**.  We've seen them enough to have an intuitive feel for what they are, but let's try to refine our understanding of them.

An **Object** is a collection of one or more related values which are all accessed using a common reference (*i.e.*, a **variable name**).  For example, a **list** object may contain multiple items, but it may also have various **attributes**, such as the length of the list.

If an object is a variable value, then a **Class** is the **definition** that dictates how the object is created, accessed, and interacts with other variables.

* For example, `message = "hello"` defines a variable `message`, and it is a string.
* The **type** is `str`, so `str` is the **class** definition.
* We know that we can do things like `message += 'foo'`.
* That is because the **`str` class** defined behavior for the `+=` operator.
* If `str` is the **type**, then we would call `message` an **instance of** `str`.

Python's documentation for functions can be found here: https://docs.python.org/3.6/tutorial/classes.html

A **Class** is defined in a code block, beginning with the keyword `class`.  You, the programmer, can come up with a useful, descriptive type name, similar to coming up with a variable name.  It is **common in industry** to **capitalize** custom class names, although it is not required by the language itself.

Here is a bare-bones example of creating a (custom) class that does not do anything (yet):


In [ ]:
# We are defining a class named `Point`.
# Eventually, it will represent a point in a
# cartesian plane, with an `x` and a `y` value.
# For now, it does nothing, as signified by
# the `pass` keyword.


class Point:
    pass


p = Point()

print(type(p))
print()

print('p is Point   :', type(p) is Point)
print('p is list    :', type(p) is list)
print('p is dict    :', type(p) is dict)
print('p is not list:', type(p) is not list)


In the preceding example, we see that creating an instance of type `Point` (or, more commonly put, **instantiating** a `Point`) looks much like calling a function.

As we have seen in the past, we can test the type of a variable using the `in` and `not in` syntax, which evaluates to a boolean value.

Compare this with the instantiation of a dictionary, and observe the similarity.

In [ ]:
# A dictionary can be constructed in several ways.
# The second example's syntax is similar to that of
# the Point class.
d = {}
d = dict()

print(type(d))
print()

print('d is Point   :', type(d) is Point)
print('d is list    :', type(d) is list)
print('d is dict    :', type(d) is dict)
print('d is not list:', type(d) is not list)


## Constructors

A **Constructor** is a **method** is called automatically when an object is created.

In general, **constructors** are a concept from Computer Science, and the idea of objects exists in many different programming languages.

In Python, the constructor is always named **`__init__`**.  You may remember that these types of names are called **dunder** methods or attributes, depending on the context.

You may also remember that **methods** are simply **functions** that are attached to a variable, or, more precisely, attached to an **object**.

**How are methods defined?**

Simple: Methods are created by putting a function definition inside the `class` definition.

The first method that we care about is the **constructor**.  As already stated, it's name is `__init__`, and it will be called automatically whenever an object is instantiated.  Let's see this in action:

In [ ]:
# We will explain `self` part in a moment.


class Point:

    def __init__(self):
        print("The constructor was called!!!!")
        print()


print("Not doing anything yet.")
print()

p = Point()
q = Point()

print("The object was created.")

r = Point()


You may be wondering why there is an argument called **`self`** in the constructor.  Let's see why we need it!

## Attributes

Now we will make the `Point` class do something interesting.  As stated earlier, we will think of this "point" as representing an `(x, y)` coordinate in a cartesian plane.  Because each point has it's own `x` and `y` value, we need to store this information in some way as part of the object.

We want to store the `x` and `y` values as **attributes** on the object.  You can think of an attribute as a variable that is attached to the main object.

Let's store the `x` value in an attribute called **`xpos`**, and the `y` value in an attribute called **`ypos`**.

This is where the argument called **`self`** comes in.  **`self`** represents the instantiated object.

As the programmer, you do not pass in anything for the `self` argument; Python will do that for you.

Consider the following modification to the `Point` class, which adds a few more arguments to the constructor.  The constructor uses the additional arguments to set the `xpos` and `ypos` attributes.

In [ ]:


class Point:

    def __init__(self, x=0, y=0):
        # Save the x and y values as attributes on this object.
        self.xpos = x
        self.ypos = y


In the next example, notice that the 2 arguments are passed to the `__init__` constructor.  In the next line, we can see that the `xpos` and `ypos` attributes have most definitely been set on the `p` object.

In [ ]:
p = Point(3, 4)
print(f"p.xpos: {p.xpos}")
print(f"p.ypos: {p.ypos}")


In the next example, notice that each instance of the Point object (variables `p`, `q`, and `r`) has it's own `xpos` and `ypos`.

Also demonstrated is the fact that the `x` and `y` arguments of the constructor have **default values**.


In [ ]:
p = Point(3, 4)
print(f"p.xpos: {p.xpos}")
print(f"p.ypos: {p.ypos}")
print()

q = Point()
print(f"q.xpos: {q.xpos}")
print(f"q.ypos: {q.ypos}")
print()

r = Point(y=42, x=-13)
print(f"r.xpos: {r.xpos}")
print(f"r.ypos: {r.ypos}")



### Observation

Look at the following line:

```print("r is", r.xpos, r.ypos)```

Isn't it ugly?

I would rather be able to type something like this:

```print("r is", r)```

Let's try it!

In [ ]:
r = Point(y=42, x=-13)

print("r is", r)

That's not very helpful!

It's not **wrong**, it's just not very useful, and it's definitely not what you would expect!

Python allows us to change this representation using a dunder method, so set's change this to something useful!

## `.__str__()`

Python has quite a few tricks for allowing our custom classes to interact in sophisticated ways.  Most often, Python supports this advanced behavior by indicating various **dunder methods** that the programmer can then implement in their custom class definitions.

The `.__str__()` dunder method is used in three ways.

1. It is invoked when the object is the argument of the `str()` function.
2. It is invoked when the object is used inside a format string.
3. It is invoked when the object is passed to the `print()` function.

The **first argument** of **all** class methods will be `self`, which is how you access the object attributes.

In [ ]:


class Point:

    def __init__(self, x=0, y=0):
        self.xpos = x
        self.ypos = y
        
    def __str__(self):
        return f"({self.xpos}, {self.ypos})"



In [ ]:
# Example of the three ways that .__str__() is automatically
# invoked.
r = Point(y=42, x=-13)

message = "r is " + str(r)
print(message)
print()

print(f"r is {r}")
print()

print("r is", r)
print()

myList = ['a', 'foo', 42]
print(f"myList is {myList}")


**A quick word about `__str__`:**

The general philosophy is that `__str__` should return a string which is easily readable and user-friendly.  In the case of a `Point` class object, it makes sense that what is printed out looks like the ordered pair of `x` and `y` coordinates.

Observe the behavior of the various types as they are printed out:

In [ ]:
# Notice what is printed out:

print('Printing a list:')
ex1 = [1,2,3]
print(ex1)
print()

print('Printing a tuple:')
ex2 = (1,2,3,4,5)
print(ex2)
print()

print('Printing a dictionary:')
ex3 = {'foo': 1, 'bar': 42, 'baz': None}
print(ex3)
print()

print('Printing the dictionary keys object:')
print(ex3.keys())
print()

print('Printing a Point:')
r = Point(y=42, x=-13)
print(r)

Again, to be clear, the `__str__` dunder method is to make your Python code more intuitive and user-friendly.

In [ ]:
r = Point(y=42, x=-13)
print(f"There is a treasure at coordinates {r}!")

## Custom Methods

Methods are simply functions that are attached to the object.  A method will usually do something or return some value that is specific to the object.

You have already been using methods on objects... ***a lot!!!***  Here are a few examples:

In [ ]:
s = "The quick brown fox jumps over the lazy dog."

print(s)

print(s.upper())
print(s.title())
print(s.lower())
print(s.islower())
print(s.replace(' ', '~'))
print(s.split())

In Python, all methods on an object are declared as part of the class.

In the next example, we will add a method called `.magnitude()` to the `Point` class.  The method will return the distance that the point is away from the origin, using the familiar Pythagorean theorem.

As you can see, adding a custom method is as simple as defining a function within the class definition.

Just like `__init__` and `__str__`, the method's first argument is always **`self`**, which is a reference to the object on which the method was called.

Inside the function, **`self`** will refer to the calling object.

In [ ]:
import math


class Point:

    def __init__(self, x=0, y=0):
        self.xpos = x
        self.ypos = y
        
    def __str__(self):
        return f"({self.xpos}, {self.ypos})"
        
    def magnitude(self):
        return math.sqrt(self.xpos**2 + self.ypos**2)



It is customary to separate methods within a class by a single blank line, and surround the entire class definition with 2 blank lines.

There is even **[official documentation](https://www.python.org/dev/peps/pep-0008/#blank-lines)** discussing this spacing convention!

In [ ]:
p = Point(3, 4)
print(f"The Cartesian location {p} has magnitude {p.magnitude()}")

q = Point()
print(f"The Cartesian location {q} has magnitude {q.magnitude()}")

r = Point(y=42, x=-13)
print(f"The Cartesian location {r} has magnitude {r.magnitude()}")

### Example of using a Point for something practical:

Here, let's generate a list of points with `x` values of 0 through pi, and `y` values as the `sin(x)`.  There will be 101 numbers total, from 0 to pi.

In [ ]:
import math
points = [Point(i * math.pi / 100, math.sin(i * math.pi / 100)) for i in range(101)]

for index, point in enumerate(points):
    print(f"{index:>3} {point}")
print()

# Of course, we can print individual points:
print(f"The last point has an `x` value of {points[-1].xpos} and a `y` value of {points[-1].ypos}.")

Notice that points[-1] is not a tuple, even though it looks like one when printed out!

In [ ]:
print(points[-1])


What happens if you try to treat it like a tuple?

Try it and find out!


In [ ]:
print(points[-1])
print(points[-1][0])


As you can see, it's not a tuple, and we can't access its attributes using the square bracket notation.

We can, however, access the attributes using their respective names.

In [ ]:
print(points[-1].xpos)
print(points[-1].ypos)

## `.__str__()` And `.__repr__()`

Python defines one additional method to use when describing an object: `.__repr__()`.

`.__repr__()` is similar to `.__str__()`, but they are used in different ways and have different reasons for existing.

Before we delve into `.__repr__()`, let's remind you of how we use `.__str__()`.

In [ ]:
# Notice the output when `__repr__()` is not defined for the class
import math


class Point:

    def __init__(self, x=0, y=0):
        self.xpos = x
        self.ypos = y
        
    def __str__(self):
        return f"({self.xpos}, {self.ypos})"
    
    def magnitude(self):
        return math.sqrt(self.xpos**2 + self.ypos**2)


p = Point(3.4, 5)
print(f"The object `p` represents the coordinate {p}.")
print()

Obviously, **`.__str__()`** is used when converting the object to a string, whether by **(1)** format string, **(2)** `str()` cast, or **(3)** `print()` function use.

Consider this use, though:

In [ ]:
points = [Point(i * math.pi / 100, math.sin(i * math.pi / 100)) for i in range(101)]
print(points)

### Wow!  That's completely useless!!!  What's happening?!?

Simple.  When a container (like the `list` that we are using above) prints out an object, it doesn't use the `.__str__()` method!  It uses one called `.__repr__()` instead, and if that is not defined, then it prints out the ugly format seen above.

Let's see what happens when we supply a `.__repr__()` method:


In [ ]:
# Now that `__repr__()` is defined, the output is nicer.
# __repr__ output should normally be in the form of a valid
# initialization of the object.
import math


class Point:

    def __init__(self, x=0, y=0):
        self.xpos = x
        self.ypos = y
        
    def __str__(self):
        return f"({self.xpos}, {self.ypos})"
    
    def __repr__(self):
        return f"Point({self.xpos},{self.ypos})"
        
    def magnitude(self):
        return math.sqrt(self.xpos**2 + self.ypos**2)


p = Point(3.4, 5)
print(f"The object `p` represents the coordinate {p}.")
print()

It doesn't change the way that the `.__str__()` works.

Let's see how it behaves when the `Point`s are in a list:

In [ ]:
points = [Point(i * math.pi / 100, math.sin(i * math.pi / 100)) for i in range(101)]
print(points)

That's a lot to look at!  Let's simplify it a bit:

In [ ]:
p = Point(3.4, 5)

print('Printing p by itself:', p)
print()

print('Printing p in a list:', [p])

When `p` is in a list, then the `.__repr__()` method is used!

Just like there is a `str()` function that calls the `.__str__()` method, there is also a `repr()` function that calls the `.__repr__()` method!

In [ ]:
p = Point(3.4, 5)

print(str(p))
print()

print(repr(p))

**But why are they different?  Why even bother with two different ways to output the strings?**

The reason is that they serve two different purposes.

1. **`.__str__()`** exists to provide a user-friendly disply of information.
2. **`.__repr__()`** exists to provide an exact (reproducible) representation of the object.  The idea is that you should be able to copy-and-paste the output from `.__repr__()` and that will serve to instantiate an object of equivalent value.

In [ ]:
# Copy-and-paste the `repr()` output from the preceding example:

newPoint = Point(3.4,5)# paste here #

print(newPoint)


It should be noted that some objects (like **lists**, seen below) have the exact same output for both `str()` and `repr()`.  This is perfectly fine, so long as the user understands what is being displayed.

In [ ]:
l = [1,2,3]

print(str(l))
print(repr(l))

### Recap:

The general philosophy is that `__repr__` should return a string which, if copy-and-pasted, could be used to create another object of equivalent values.

#### What's the difference between `.__str__()` and `.__repr__()`?

`.__str__()` should make your object simple to grasp.  ***Readability*** is the goal.

`.__repr__()` should be exact in its meaning.  ***Unanbiguity*** is the goal.

`.__str__()` and `.__repr__()` are called automatically by the `str()` and `repr()` functions, respectively.

But they are also just **methods** and can be called direcly, even though it is not common to do so.

In fact, **please don't do this**.  It's ugly.  It's an example of "just because you can doesn't mean that you should."

In [ ]:
# Sometimes the `__str__` and the `__repr__` methods are the same.
p = Point(3.4, 5)

print(p.__str__())
print()

print(p.__repr__())

Let's see an example of an object whose `str()` and `repr()` are quite different.  Although we have not seen this type of object before, it's use should be obvious: It is an object that represents a specific point in time (date and time, to be exact).

Here, you see the philosophy of the two methods exemplified: **Readability vs. Unambiguity**

In [ ]:
# Here is an example of an object whose str() and repr()
# are different.
import datetime
today = datetime.datetime.now()
print(str(today))
print(repr(today))

## Documentation

You have already seen that functions can have documentation, and that documentation string is stored on the `.__doc__` attribute.

In [ ]:
def doubleIt(x):
    '''
    Returns a value that is double that of the supplied argument.
    
    Arguments:
    x - The value to be doubled.
    '''

    return x * 2

Accessing the documentation using the `help()` function:

In [ ]:
help(doubleIt)

Accessing the documentation using the `.__doc__` dunder attribute.

In [ ]:
print(doubleIt.__doc__)

In the same way, **classes** can have documentation, as can all of their methods.

Consider the following defintion of a `Fraction` class:

In [ ]:


class Fraction:

    def __init__(self, numerator=0, denominator=1):
        self.n = numerator
        self.d = denominator

    def __str__(self):
        return f"{self.n}/{self.d}"

    def __repr__(self):
        return f"Fraction({self.n},{self.d})"


help(Fraction)

As you can see, Python provides default documentation for the class and it's methods.  It even includes some that provide built-in functionality (attributes) that we have not seen before!

But the documentation is not very helpful.  Let's do better  (Notice the spacing provided for readability, discussed earlier in this section):

In [ ]:


class Fraction:
    """
    Represents a mathematical fraction, with integer numerator and
    denominator values.
    
    Constructor:
      Fraction() -> create a new fraction object
      Fraction(numerator) -> create a new fraction object with the value
        <numerator>/1
      Fraction(numerator, denominator) -> create a new fraction object
        with the value <numerator>/<denominator>
    """

    def __init__(self, numerator=0, denominator=1):
        '''
        Construct a Fraction object.  The denominator should never be 0.
        
        Arguments:
        numerator - The numerator of the Fraction.  Defaults to 0.
        denominator - The denominator of the Fraction.  Defaults to 0.
        '''
        self.n = numerator
        self.d = denominator

    def __str__(self):
        '''
        Return a string representation of the Fraction.
        May be improper.
        '''
        return f"{self.n}/{self.d}"

    def __repr__(self):
        '''
        Return a string representation of a constructor for the Fraction.
        '''
        return f"Fraction({self.n},{self.d})"


help(Fraction)

As you can see, good documentation goes a long way to helping the programmer know how to use and what to expect from your code, including classes.

You should **always** write documentation for your code.  You **will** forget how to use it at some point in the future, and documentation is how you minimize the pain of revisiting code written a long time ago.

You may notice that there is actually **more** documentation than code in the preceding `Fraction` class example.  This is both intended and, in some way, desireable.  It's often said that documentation is more important than the code itself.  If this is the case, then make it a practice to **always** write good comments for anything you write.

Consider documentation to be like hygiene.  Everybody appreciates it when it's good.  When it's bad, nobody wants to work with you.

**Practice good code hygiene.  Write good documentation.**

## More Dunder Methods

We can make our `Fraction` class more useful and more intuitive for programmers to use.  Let's start by adding the ability to "add" two fractions together.

We will define a method called `.add()`, which takes another `Fraction` object as an argument.  The original object's value is not changed, just as adding `3 + 5` will not alter the value of the `3` or the `5`, but evaluates to a new value of `8`.

In [ ]:


class Fraction:
    """
    Represents a mathematical fraction, with integer numerator and
    denominator values.
    
    Constructor:
      Fraction() -> create a new fraction object
      Fraction(numerator) -> create a new fraction object with the value
        <numerator>/1
      Fraction(numerator, denominator) -> create a new fraction object
        with the value <numerator>/<denominator>
    """

    def __init__(self, numerator=0, denominator=1):
        '''
        Construct a Fraction object.  The denominator should never be 0.
        
        Arguments:
        numerator - The numerator of the Fraction.  Defaults to 0.
        denominator - The denominator of the Fraction.  Defaults to 0.
        '''
        self.n = numerator
        self.d = denominator

    def __str__(self):
        '''
        Return a string representation of the Fraction.
        May be improper.
        '''
        return f"{self.n}/{self.d}"

    def __repr__(self):
        '''
        Return a string representation of a constructor for the Fraction.
        '''
        return f"Fraction({self.n},{self.d})"

    def add(self, other):
        '''
        Return a new Fraction object whose value is equivalent to adding
        the current object with `other`.
        
        Arguments:
        other - The value to be added to the current object's value.
        '''
        return Fraction((self.n * other.d) + (self.d * other.n), self.d * other.d)



In [ ]:
f = Fraction(1,3)
g = Fraction(2,5)

print("f and g original values:")
print(f"f is {f}")
print(f"g is {g}")
print()


print(f"f + g = {f.add(g)}")
print()

print("f and g are unchanged:")
print(f"f is {f}")
print(f"g is {g}")
print()


Using a `.add()` method is a bit cumbersome.  Let's try to make this look a bit more "natural", using a `+`.

In [ ]:
f = Fraction(1,3)
g = Fraction(2,5)

print("f and g original values:")
print(f"f is {f}")
print(f"g is {g}")
print()

print(f"f + g = {f + g}")


**Oops!**

As you can see, Python doesn't know what to do with this syntax.  Python doesn't "know" anything about our custom class, including how to perform an addition.

Thankfully, Python allows us to define behavior for all of the mathematical operators.  That includes not only `+ - * / // % **`, but also the shortcut operators `+= -= *= /= %= **= //=`.  It also includes the comparison operators `< <= > >= == !=`.

Basic documentation can be found here: https://docs.python.org/3.6/reference/datamodel.html#emulating-numeric-types

Emulating the `+` operation is easy: define the `.__add__()` dunder method.

When Python sees something like `f + g`, this is what it does:

1. Python examines the type of `f`.  In this case, it is a `Fraction` class type.
2. Python looks to see whether or not the `.__add__()` method is defined on that class.
3. Python calls `f.__add__(g)`.

Earlier, we wrote an `.add()` method for our `Fraction` class.  Now, let's rename it to be `.__add__()`, and then see if the `f + g` expression works!

In [ ]:


class Fraction:
    """
    Represents a mathematical fraction, with integer numerator and
    denominator values.
    
    Constructor:
      Fraction() -> create a new fraction object
      Fraction(numerator) -> create a new fraction object with the value
        <numerator>/1
      Fraction(numerator, denominator) -> create a new fraction object
        with the value <numerator>/<denominator>
    """

    def __init__(self, numerator=0, denominator=1):
        '''
        Construct a Fraction object.  The denominator should never be 0.
        
        Arguments:
        numerator - The numerator of the Fraction.  Defaults to 0.
        denominator - The denominator of the Fraction.  Defaults to 0.
        '''
        self.n = numerator
        self.d = denominator

    def __str__(self):
        '''
        Return a string representation of the Fraction.
        May be improper.
        '''
        return f"{self.n}/{self.d}"

    def __repr__(self):
        '''
        Return a string representation of a constructor for the Fraction.
        '''
        return f"Fraction({self.n},{self.d})"

    def __add__(self, other):
        '''
        Return a new Fraction object whose value is equivalent to adding
        the current object with `other`.
        
        Arguments:
        other - The value to be added to the current object's value.
        '''
        return Fraction((self.n * other.d) + (self.d * other.n), self.d * other.d)



In [ ]:
f = Fraction(1,3)
g = Fraction(2,5)

print("f and g original values:")
print(f"f is {f}")
print(f"g is {g}")
print()

print(f"f + g = {f + g}")


**It worked!!**

What happens if we try to add an integer instead?

In [ ]:
f = Fraction(1,3)
print(f + 3)

It didn't work. :(

What happened?  Well, when Python saw `f + 3`, Python tried to call `f.__add__(3)`.  Inside the `.__add__()` method, the `other` argument is assigned the value `3`.  Because `other` is an `int`, it has no `.d` or `.n` attribute, and hence the error.

We can protect against the error by writing special code for handling the `int` by converting it to a `Fraction` object first.

While we are at it, we should make sure that the `.__add__()` method handles any other type of possible input, too.  If the type is not specified, then we should return a special value: `NotImplemented`.

In [ ]:


class Fraction:
    """
    Represents a mathematical fraction, with integer numerator and
    denominator values.
    
    Constructor:
      Fraction() -> create a new fraction object
      Fraction(numerator) -> create a new fraction object with the value
        <numerator>/1
      Fraction(numerator, denominator) -> create a new fraction object
        with the value <numerator>/<denominator>
    """

    def __init__(self, numerator=0, denominator=1):
        '''
        Construct a Fraction object.  The denominator should never be 0.
        
        Arguments:
        numerator - The numerator of the Fraction.  Defaults to 0.
        denominator - The denominator of the Fraction.  Defaults to 0.
        '''
        self.n = numerator
        self.d = denominator

    def __str__(self):
        '''
        Return a string representation of the Fraction.
        May be improper.
        '''
        return f"{self.n}/{self.d}"

    def __repr__(self):
        '''
        Return a string representation of a constructor for the Fraction.
        '''
        return f"Fraction({self.n},{self.d})"

    def __add__(self, other):
        '''
        Return a new Fraction object whose value is equivalent to adding
        the current object with `other`.
        
        Arguments:
        other - The value to be added to the current object's value.
        '''
        if type(other) is int:
            return self + Fraction(other)

        elif type(other) is Fraction:
            return Fraction((self.n * other.d) + (self.d * other.n), self.d * other.d)

        else:
            return NotImplemented



In [ ]:
f = Fraction(1,3)
g = Fraction(2,5)
print(f + g)
print(f + 3)

Now, let's see the `NotImplemented` in action:

In [ ]:
f = Fraction(1,3)
g = Fraction(2,5)
print(f + .5)

That is what we want to happen!  The code gives an appropriate error message, which is helpful to the programmer.

Let's see what happens when we reverse the order of the operators:

In [ ]:
f = Fraction(1,3)
print(f + 3)
print(3 + f)

As you can see, it doesn't work when we reverse the order.  And why is that?  Consider what happense why Python sees `3 + f`.

1. Python examines the type of `3`.  In this case, it is an `int` type.
2. Python looks to see whether or not the `.__add__()` method is defined on that type.
3. An `int` is not a class *per se*, but Python takes a similar course of action in trying to add the `f` to the `3`.

Because an `int` doesn't know anything about the `Fraction` type, it can't perform the addition.

Thankfully, Python provides a way to meet this need.  It defines a "reverse addition" dunder method (aptly named `.__radd__()`) in the event that the normal method does not work.

In [ ]:


class Fraction:
    """
    Represents a mathematical fraction, with integer numerator and
    denominator values.
    
    Constructor:
      Fraction() -> create a new fraction object
      Fraction(numerator) -> create a new fraction object with the value
        <numerator>/1
      Fraction(numerator, denominator) -> create a new fraction object
        with the value <numerator>/<denominator>
    """

    def __init__(self, numerator=0, denominator=1):
        '''
        Construct a Fraction object.  The denominator should never be 0.
        
        Arguments:
        numerator - The numerator of the Fraction.  Defaults to 0.
        denominator - The denominator of the Fraction.  Defaults to 0.
        '''
        self.n = numerator
        self.d = denominator

    def __str__(self):
        '''
        Return a string representation of the Fraction.
        May be improper.
        '''
        return f"{self.n}/{self.d}"

    def __repr__(self):
        '''
        Return a string representation of a constructor for the Fraction.
        '''
        return f"Fraction({self.n},{self.d})"

    def __add__(self, other):
        '''
        Return a new Fraction object whose value is equivalent to adding
        the current object with `other`.
        
        Arguments:
        other - The value to be added to the current object's value.
        '''
        if type(other) is int:
            return self + Fraction(other)

        elif type(other) is Fraction:
            return Fraction((self.n * other.d) + (self.d * other.n), self.d * other.d)

        else:
            return NotImplemented

    def __radd__(self, other):
        '''
        Return a new Fraction object whose value is equivalent to adding
        the current object with `other`.
        
        Arguments:
        other - The value to be added to the current object's value.
        '''
        return self + other



In [ ]:
f = Fraction(1,3)
g = Fraction(2,5)
print(f"f + g = {f + g}")
print(f"f + 3 = {f + 3}")
print(f"3 + f = {3 + f}")

It worked!

In this case, `.__radd__()` falls back on the behavior of `.__add__()`, and because `.__add__()` is implemented correctly for any type (and the little detail that `+` is commutative), then we know that `.__radd__()` is also implemented correctly.

How about the **`+=`** operator?  That is easy to implement as well!

First, think about what `+=` does.  `x += 3` is a shorthand for `x = x + 3`.  There are 2 important things to consider.

1. `+=` can use the `+` syntax that we have already defined.
2. `+=` will change the value of `x`, which is a different behavior than the `+` operator.

`+=` is implemented using the `.__iadd__()` dunder method.

In [ ]:


class Fraction:
    """
    Represents a mathematical fraction, with integer numerator and
    denominator values.
    
    Constructor:
      Fraction() -> create a new fraction object
      Fraction(numerator) -> create a new fraction object with the value
        <numerator>/1
      Fraction(numerator, denominator) -> create a new fraction object
        with the value <numerator>/<denominator>
    """

    def __init__(self, numerator=0, denominator=1):
        '''
        Construct a Fraction object.  The denominator should never be 0.
        
        Arguments:
        numerator - The numerator of the Fraction.  Defaults to 0.
        denominator - The denominator of the Fraction.  Defaults to 0.
        '''
        self.n = numerator
        self.d = denominator

    def __str__(self):
        '''
        Return a string representation of the Fraction.
        May be improper.
        '''
        return f"{self.n}/{self.d}"

    def __repr__(self):
        '''
        Return a string representation of a constructor for the Fraction.
        '''
        return f"Fraction({self.n},{self.d})"

    def __add__(self, other):
        '''
        Return a new Fraction object whose value is equivalent to adding
        the current object with `other`.
        
        Arguments:
        other - The value to be added to the current object's value.
        '''
        if type(other) is int:
            return self + Fraction(other)

        elif type(other) is Fraction:
            return Fraction((self.n * other.d) + (self.d * other.n), self.d * other.d)

        else:
            return NotImplemented

    def __radd__(self, other):
        '''
        Return a new Fraction object whose value is equivalent to adding
        the current object with `other`.
        
        Arguments:
        other - The value to be added to the current object's value.
        '''
        return self + other

    def __iadd__(self, other):
        '''
        Add the value of `other` to the current Fraction object.

        Arguments:
        other - The value to be added to the current object's value.
        '''
        self = self + other
        return self


help(Fraction)

In [ ]:
f = Fraction(1,3)

print(f"f is {f}")
print()

f += 3

print("After f += 3, ")
print(f"f is {f}")


In [ ]:


class Fraction:
    """
    Represents a mathematical fraction, with integer numerator and
    denominator values.
    
    Constructor:
      Fraction() -> create a new fraction object
      Fraction(numerator) -> create a new fraction object with the value
        <numerator>/1
      Fraction(numerator, denominator) -> create a new fraction object
        with the value <numerator>/<denominator>
    """

    def __init__(self, numerator=0, denominator=1):
        '''
        Construct a Fraction object.  The denominator should never be 0.
        
        Arguments:
        numerator - The numerator of the Fraction.  Defaults to 0.
        denominator - The denominator of the Fraction.  Defaults to 0.
        '''
        self.n = numerator
        self.d = denominator

    def __str__(self):
        '''
        Return a string representation of the Fraction.
        May be improper.
        '''
        return f"{self.n}/{self.d}"

    def __repr__(self):
        '''
        Return a string representation of a constructor for the Fraction.
        '''
        return f"Fraction({self.n},{self.d})"

    def __add__(self, other):
        '''
        Return a new Fraction object whose value is equivalent to adding
        the current object with `other`.
        
        Arguments:
        other - The value to be added to the current object's value.
        '''
        if type(other) is int:
            return self + Fraction(other)

        elif type(other) is Fraction:
            return Fraction((self.n * other.d) + (self.d * other.n), self.d * other.d)

        else:
            return NotImplemented

    def __radd__(self, other):
        '''
        Return a new Fraction object whose value is equivalent to adding
        the current object with `other`.
        
        Arguments:
        other - The value to be added to the current object's value.
        '''
        return self + other

    def __iadd__(self, other):
        '''
        Add the value of `other` to the current Fraction object.

        Arguments:
        other - The value to be added to the current object's value.
        '''
        self = self + other
        return self
    
    def __ge__(self, other):
        if type(other) is float:
            return (self.n / self.d) >= other
        
        elif type(other) is int:
            return self >= Fraction(other)
        
        elif type(other) is Fraction:
            return (self.n * other.d) >= (other.n * self.d)
        
        return NotImplemented
    
    def __le__(self, other):
        if type(other) is float:
            return (self.n / self.d) <= other
        
        elif type(other) is int:
            return self <= Fraction(other)
        
        elif type(other) is Fraction:
            return (self.n * other.d) <= (other.n * self.d)
        
        return NotImplemented


#help(Fraction)

In [ ]:
f = Fraction(1,3)
g = .1

print(f"The statement {f} >= {g} is {f>=g}")
print(f"The statement {g} >= {f} is {g>=f}")

print()
print(f)


## The Main Takeaways

1. By adding more and more dunder methods, you can make your custom class interact with the rest of your program seamlessly and intuitively.
2. Classes and Objects are **tools**... **mental models** to help us think through and express ideas.  In our examples, we have used classes to express `Point` and `Fraction` ideas.  Python provides classes to represent [date and time](https://docs.python.org/3.6/library/datetime.html), [zip files](https://docs.python.org/3.6/library/zipfile.html), [variable caching](https://docs.python.org/3.6/library/pickle.html), [database connections](https://docs.python.org/3.6/library/sqlite3.html), [Parsing HTML](https://docs.python.org/3.6/library/markup.html), [manipulating multimedia files](https://docs.python.org/3.6/library/mm.html), and even [data visualizations](https://matplotlib.org/).
3. Read documentation to learn how to use libraries and their various classes.  If the libraries are designed well, then they will use consistent styles and patterns of code.  This is commonly referred to as being **pythonic**.  Developers often talk about writing **pythonic**, or **idiomatic**, code.

# Random Numbers

In Computer Science, random numbers come in 2 types: **true random** and **pseudorandom**.

These two types of random numbers serve different purposes for programmers.  You may think that numbers should always be *completely* random, and sometimes that is true, but sometimes a programmer needs them to be *predictably* random.

*Completely* random numbers are considered to be **true random**, and is used for cryptographic security.  But computers are **deterministic** machines, so true randomness is hard and slow to generate.

Most programs use **pseudorandom** numbers.  They are fast to generate, and "random enough" for most uses.  Pseudorandom numbers use a "seed" as a starting point.  The reason that we call these "predictably random" is that the sequence of random numbers that are generated are always the same, given the same starting seed.  Python can use the system clock to automatically set the seed.  Of course, this means that if two computers begin a script at the exact same moment, then they will generate the exact same sequence of random numbers.

We will focus on **pseudorandom** numbers.  Python generates random numbers using the `random` library (https://docs.python.org/3.6/library/random.html).

Important functions:

* `random.seed()` - Sets a seed value for the pseudo-random number generation.
* `random.randrange()` - Get a (pseudo) random integer value.  This function has 3 possible parameters, which are conceptually similar to the `range()` function.
  * `random.randrange(10)` will return a random integer between 0 and 9 (inclusive).
  * `random.randrange(5, 10)` will return a random integer between 5 and 9 (inclusive).
  * `random.randrange(5, 10, 2)` will return a random integer between 5 and 9 (inclusive), but will have a "skip" of 2.  That means that the possible numbers will be 5, 7, or 9.
* `random.random()` - Get a (pseudo) random floating point number, 0 <= x < 1.

In [ ]:
# Printing 10 random integers

import random

for x in range(10):
    print(random.randrange(1000))

In [ ]:
for x in range(10):
    print(random.randrange(1000))

In [ ]:
# Intentionally setting the seed to 42:
random.seed(42)
for x in range(10):
    print(random.randrange(1000))

In the next 2 cells, we'll set the seed to `42`, and then generate 10 numbers:

In [ ]:
# Intentionally setting the seed to 42:
random.seed(42)
for x in range(10):
    print(random.randrange(1000))

In [ ]:
# Intentionally setting the seed to 42:
random.seed(42)
for x in range(10):
    print(random.randrange(1000))

As you can see, setting the same seed value results in an identical sequence of numbers.

Setting the same seed over and over results in the same number being generated repeatedly.

In [ ]:
for x in range(10):
    random.seed(42)
    print(random.randrange(1000))

Generally speaking, the seed is only set once when a script is run, and most programmers put it towards the beginning.

In this next example, we will let Python use the system clock to set the seed.

In [ ]:
random.seed()
for x in range(10):
    print(random.randrange(1000))

So far, we have only generated integers.  Floating point numbers are a bit different.

`random.random()` will generate a floating point number `>= 0` and `< 1`.

In [ ]:
for _ in range(10):
    print(random.random())

If you want a different **range**, then you **scale** the floating point number by multiplying it.

See this next example, which will generate 10 numbers between 0 and 50:

In [ ]:
for _ in range(10):
    print(random.random() * 50)

Of course, random numbers can be generated in comprehensions, too.

In [ ]:
randomNumbers = [random.random() for _ in range(10)]

print(randomNumbers)

You can use a bit more mathematical manipulation to get more complex ranges.

This next example will generate a list of floating point numbers between 10 and 50:

In [ ]:
randomNumbers = [(random.random() * 40) + 10 for _ in range(10)]

print(randomNumbers)

# Numpy
An alternative (more expressive) way to deal with numbers.
* Numpy documentation: https://docs.scipy.org/doc/
* Numpy Tutorial with Examples: https://www.pythonprogramming.in/numpy-tutorial-with-examples-and-solutions.html
* Numpy tutorial: https://docs.scipy.org/doc/numpy/user/quickstart.html
* https://github.com/enthought/Numpy-Tutorial-SciPyConf-2018

## What Is Numpy, And Why Do We Need It?

Python lists are heterogeneous.  Python is *flexible*, and often that is a really, rally good thing!  But the tradeoff of that flexibility is that Python is also slow.  Worse yet, it can even be unintuitive or overly verbose in some situations!

In [ ]:
# Python lists:
l = [1, 2, 3]
print(l)

In [ ]:
# but also
l = [1, 'a', True]
print(l)

Do you remember what happens when you try to do math with lists?

In [ ]:
# Let's add two lists in Python.
a = [1, 2, 3]
b = [10, 11, 12]
print (a + b)

Wait, that's not what we want to do!

 We want it to add the two lists together by element. That is, the output should be `[11, 13, 15]`

We can do that in Python, but it takes a bit more work.

In [ ]:

a = [1, 2, 3]
b = [10, 11, 12]

print(f"`a` is: {a}")
print(f"`b` is: {b}")
print()

print("`a` and `b` zipped together:")
print([*zip(a,b)])
print()

print("Using `zip()` and a list comprehension to add together the list elements:")
print([x + y for (x,y) in zip(a,b)])


Let's try do to do the same thing in numpy.

Of course, we need to import the `numpy` library.

In [ ]:
# It is customary to alias numpy as `np`.
import numpy as np

a = np.array([1, 2, 3])
b = np.array([10, 11, 12])

print(f"`a` is: {a}")
print(f"`b` is: {b}")
print()

print(f"`a` + `b` is: {a + b}")

First, notice how the numpy `.__str__()` method behaves.  It prints out the numbers in square brackets, but there are no commas.

What happens if we ask for it's `.__repr__()`?

In [ ]:
# It is customary to alias numpy as `np`.
import numpy as np

a = np.array([1, 2, 3])
b = np.array([10, 11, 12])

print(f"`a` is: {repr(a)}")
print(f"`b` is: {repr(b)}")
print()

print(f"`a` + `b` is: {repr(a + b)}")

We can see that this object is called an `array`, and that its value can be duplicated using a list of numbers.

Let's get back to that addition that we saw earlier.

It definitely OK, so it's a bit more intuitive.  Is it faster?

Let's find out with a special command in Jupyter called `%timeit`

In [ ]:
a = [1, 2, 3]
b = [10, 11, 12]
%timeit c = [*map(lambda t: t[0] + t[1], zip(a,b))]


In [ ]:
a = [1, 2, 3]
b = [10, 11, 12]
%timeit c = [x + y for (x,y) in zip(a,b)]


In [ ]:
a = np.array([1, 2, 3])
b = np.array([10, 11, 12])
%timeit c = a + b
print(a + b)

This was a simple example.  The more complex that your Python code gets, the more efficient Numpy will be as compared to pure Python.

## Basic Numpy Usage

Generally speaking, numpy creates **arrays** based on some sequence.  As you already know, there are various types of sequences in Python.

In [ ]:
# Creating an array
a = np.array(range(10))
print(a)
print()

b = np.array([1,3,5,6])
print(b)
print()

c = np.array([*"foo"])
print(np.array([*"foo"]))

In [ ]:
# See the types
print('a =', a)
print('The type of `a` is', type(a))
print('The type of the data that `a` contains is', a.dtype)
print(a[0])
print(type(a[0]))

In [ ]:
# Arrays have dimensions and shape, and other info
print('a =', a)
print('`a` has dimensions:', a.ndim)
print('`a` has shape:', a.shape)
print('Each element of a takes up %d bytes' % a.itemsize)
print('All of the data in `a` takes %d bytes total' % a.nbytes)

# Basic Numpy Math

In [ ]:
a = np.array([1,2,3,4])
b = np.array([2,3,4,5])

print(f"`a` is: {a}")
print(f"`b` is: {b}")
print()

print("a + b =", a + b)
print()

print("a * b =", a * b)
print()

print("a ** b =", a ** b)


As a side note, this lets us see something very interesting in the string output of a numpy array: Numpy will make all columns take up the same amount of space, for easier visual inspection.

In [ ]:
np.array(['a', 'b'])

## `arange()`

Numpy's `arange()` function is very similar to Python's `range()` function, with a few exceptions:
* `numpy.arange()` accept floating point numbers.
* `dtype` is derived from the start, stop, and step values.

The arguments are similar to `range()`.
* np.arange(stop)
* np.arange(start, stop)
* np.arange(start, stop, skip)

First, observe the differences between these different ways of generating a list of numbers:

In [ ]:
print([*range(10)])
print(np.array(range(10)))
print()

print(np.arange(10))
print(np.arange(10.))

See how the following examples have the same behaviors for the start and stop values:

In [ ]:
print([*range(10, 26)])
print(np.arange(10, 26))

In [ ]:
print([*range(-10, 11, 2)])
print(np.arange(-10, 11, 2))

Suppose that you wanted to get a list of floating point numbers from -1 to +1, in increments of .25.

How would you do this in Python?

In [ ]:
print([-1.0, -0.75, -0.5, -0.25, 0.0, 0.25, 0.5, 0.75, 1.0])

Of course you could hard code it, but how would you generate this list using traditional Python?  It would require a mixture of `range()` and a **comprehension**.

In [ ]:
# How would you get an equivalent list in standard Python?
print([.01 * x for x in range(-100, 101, 25)])
print()

print([x / 4 for x in range(-4, 5)])

But this is **ugly**.

Numpy's `arange()` function will accept floating point numbers for its `begin`, `end`, and `skip`, so it becomes much easier:

In [ ]:
print(np.arange(-1, 1.1, .25))
print()

# Or, if you prefer:
print(np.arange(-4, 5) / 4)

Let's consider the **type** of the array, when created with the `arange()` function.

In [ ]:
print("Creating an array of integers")
print(np.arange(11), np.arange(11).dtype)
print(np.arange(11) / 3)
print()

print("Creating an array of floats")
print(np.arange(11.), np.arange(11.).dtype)
print(np.arange(11.) / 3)

**Wait!!!!**

Did you notice that we devided the array by 3 and it didn't give us an error?

`print(np.arange(11) / 3)`

This is because numpy works much like a mathematician would expect.  Every value in the array is divided by 3!  This will also work with the other mathematical operations, too!

Also, notice that numpy changed the **type** of the array from `int32` to `float64` when the division was performed.


In [ ]:
# Numpy gives us a value for pi.
print(np.pi)

Here is an example of getting a list of 10 numbers and scaling them from 0 to 2π.  Notice that we are multiplying and dividing the entire array by several numbers in a row.

In [ ]:
nums = np.arange(11.)

print(nums * np.pi * 2 / 10)

In [ ]:
np.arange(0, (np.pi * 2) + .00001, np.pi * 2 / 10)

Again, as a reminder, compare this to the behavior of Python lists.

In [ ]:
# Compare with Python lists:
[1,2,3] * 4

In [ ]:
np.array([1,2,3]) * 4

Numpy provides a lot more mathematical functions that can be applied to an entire array at a time.  In the following example, you will see the `sin` function (from the `numpy` library, not the `math` library) being applied to an entire array.

You can find more math functions here: https://docs.scipy.org/doc/numpy/reference/routines.math.html

In [ ]:
# Example of applying a math function to every element of the list
x = np.arange(11) * np.pi * 2 / 10

print("Values for x are:")
print(x)
print()

y = np.sin(x)
print("Values for y are:")
print(y)

## Various Array Behaviors

In [ ]:
# Changing an element of an array using bracket notation and an index.

a = np.arange(4)
print("before:")
print(a)
print()

a[0] = 10
print("after:")
print(a)

In [ ]:
# Type coersion.  Notice the array type after assigning
# a value to a[0].

a = np.arange(4)
print(f"`a` is {a}")
print(f"`a`'s type is: {a.dtype}")
print()

a[0] = -15.6
print(f"now `a` is {a}")
print(f"`a`'s type is: {a.dtype}")


In [ ]:
# Filling an array (replacing the values)
a = np.arange(4)
print("before:")
print(f"`a` is {a}")
print()

a.fill(-4.8)
print("after:")
print(f"`a` is {a}")


In [ ]:
# specifically setting the array type
a = np.arange(4, dtype=np.float64)
print(f"`a` is {a}")
print(f"`a`'s type is: {a.dtype}")
print()

a.fill(-4.8)
print("after:")
print(f"`a` is {a}")


## Multidimensional Arrays

Numpy supports multidimensional arrays.  It is doubtful that you need this in run-of-the-mill programming, but specialty applications may require it.  Thankfully, Numpy makes it intuitive to work with, too.

As you saw earlier, each array has several properties that describe the array contents.

In [ ]:
a = np.array([[0,1,2,3], [10,11,12,13]])
print(a)
print("`a.shape` is", a.shape)
print("`a.size` is", a.size)
print("`a.ndim` is", a.ndim)

In [ ]:
# Notice how the standard Python `len()` knows nothing about
# the inner lists.
# The point is that Python lists (which are built-in) are simple
# and flexible, but not very sophisticated.
# Conversely, Numpy arrays are less flexible, but are much more
# powerful in expressive power and consistency in what they represent.
myList = [[0,1,2,3], [10,11,12,13]]
print(myList)
print(f"`len` of `myList` is {len(myList)}")
print(f"`len` of `myList[0]` is {len(myList[0])}")

In [ ]:
# This is the preferred (and much suggested) way to access a multi-dimensional element

a = np.array([[0,1,2,3], [10,11,12,13]])
print("before:")
print(a)
print()

a[1,3] = -999
print("after:")
print(a)

In [ ]:
# Although this way works, it is discouraged
a[1][3] = 33
print(a)

In [ ]:
# If we only ask for one item, we get everything in a row.
print(a[1])

## Slicing
It works just like you would expect!

In [ ]:
a = np.arange(10,15)
print(a)

In [ ]:
print(a[1:3])

In [ ]:
a = np.arange(20)
print(a)
print(a[3:])
print(a[3:17:4])

In [ ]:
print(a[::-1])

Slicing gets very interesting when you begin working with multidimensional data, though!

In [ ]:
print(np.arange(25))

In [ ]:
# This will use the `reshape()` method to change the
# data from a 1-dimensional array of 25 values into 
# a 2-dimensional array of 5 rows and 5 columns.

a = np.arange(25)
print("before:")
print(a)
print()

a = a.reshape(5,5)
print("after:")
print(a)

In [ ]:
np.arange(27).reshape(3,3,3)

In [ ]:
print(a)
print()

print("getting a slice:")
print(a[1, 3:5])

Notice that we asked for a specific row (`1`) and then a slice of its contents (`3:5`).

In [ ]:
print(a)
print()

print("getting a slice:")
print(a[3:, 3:])

### An aside about how columns and rows are requested.
Notice The difference between the following two ways of asking for column 2.

In [ ]:
# This example returns an array with a single dimension.  We "lost" a dimension.
a = np.arange(25).reshape(5,5)

print(a)
print()

print(a[:,2])
print(a[:,2].shape)

In [ ]:
# When using a slice, on the other hand, both dimensions are retained.
print(a)
print()

print(a[:,2:3])
print(a[:,2:3].shape)

In [ ]:
# Notice in this case, only a single number (e.g., no dimensions) is returned.
print(a[1,1])
print(a[1,1].shape)

In [ ]:
# On the other hand, this code asks for the same element, but it retains
# all the dimensions.
print(a[1:2,1:2])
print(a[1:2,1:2].shape)

The takeaway from this short series of examples is this simple realization:
  1. When we ask for a single element from a dimension, that dimension is, in effect, collapsed out of the result.
  2. Alternatively, when we as for a slice, then that dimension is retained in the result (even if it is as small as a dimension of size 1).

In [ ]:
# You can even do weird things with skips
print(a)
print()


print(a[2::2,::2])

Alternatively, you can get very specific about your selections.

In [ ]:
# Rather than a slice, we can use a list to request specific rows or columns.
# Note: Do not try to use a list to pick both rows and columns at the same time.
#       It has a very specific behavior, and is probably not what you want to happen.
print(a)
print()

print(a[[1,4],2:])

In [ ]:
# This is an advanced example, using a mask to select the rows.
# Unless you are very much interested in the subject, you can skip this.
mask = np.array([1,0,1,1,0], dtype=bool)
print(a)
print(a[mask])

Boolean masks like this get *a lot* more complicated!

## Other Common Numpy Tools

In [ ]:
# There are a couple of helper functions to create an array of a specific size.
z = np.zeros(10)
print(z)
print(z.dtype)

In [ ]:
z = np.zeros(10, dtype="int32")
z[5] = 6.5
print(z)
print(z.dtype)

In [ ]:
# Of course, you can also pass a shape as the size
o = np.ones((3,8), dtype="int32")
print(o)
print(o.dtype)

In [ ]:
# If linear algebra is your thing, you can get the identity matrix
for i in range(6):
    print(np.identity(i))
    print()

In [ ]:
# Creating an array without initializing any values:
e = np.empty(100)
print(e)

In [ ]:
f = np.empty(10)
f.fill(20)
print(f)

In [ ]:
# It's a bit slower, but this works, too:
f = np.empty(10)
f[:] = 20
print(f)

In [ ]:
%timeit f.fill(20)
%timeit f[:] = 20

In [ ]:
# linspace(start, stop, count)
# Create `count` evenly spaced numbers between start and stop, inclusive
print(np.linspace(-5,5,20))

In [ ]:
# Evidently, if you omit the `count`, it defaults to 50
print(np.linspace(1,50))
print(np.linspace(2,100))
print(np.linspace(0,3))

In [ ]:
# Of course, you might need a logarithmic spacing instead, so there is logspace()
# logspace(start, stop, count, base=10)
# Range is base**start to base**stop.  Base id defaulted to 10, count to 50.
print(np.logspace(1,3))
print()

print(np.logspace(1,10,10, base=2))
print()

print(np.logspace(1,10,10))
print()

print(np.logspace(0,2,10))

## Aggregate Functions

Just as the built-in `len()` function doesn't know how to interpret a numpy array, neither do most of the other built-in functions.  Numpy provides its own version of these functions in order to support their more sphisticated usage.

There are **a lot** of functions available, but I will only demonstrate a few here.

**Documentation:** https://numpy.org/doc/stable/reference/routines.math.html

In [ ]:
import numpy as np

# Use this as sample data
a = np.arange(-15, 15, dtype="int64").reshape(5, 6) ** 2 - 35
print(a)

As you can see, the data is two-dimensional.  We can add up the values in several ways, though, using the numpy `sum()` function.

In [ ]:
print(np.sum(a))

In [ ]:
print(a)
print()

# axis=0 sums up the columns
# axis=0 eliminates the rows
print("Summing up the columns:")
print(np.sum(a, axis=0))
print(np.sum(a, axis=0).shape)
print()

# axis=1 sums up the rows
# axis=1 eliminates the columns
print("Summing up the rows:")
print(np.sum(a, axis=1))
print(np.sum(a, axis=1).shape)


In [ ]:
# prod() multiplies everything in the matrix.
# Like sum(), it supports the `axis` argument.
print(np.prod(a))
print()

print("Multiplying the columns:")
print(np.prod(a, axis=0))
print()

print("Multiplying the rows:")
print(np.prod(a, axis=1))

In [ ]:
# Beware the nan!
b = np.sqrt(np.arange(-3,4, dtype="float64")) + 1
print(b)
print()

print(np.prod(b))
print(np.nanprod(b))
print()

print(np.sum(b))
print(np.nansum(b))

In [ ]:
# Alternative notation for the same function calls:
print(a)
print()

print(np.sum(a))
print(a.sum())
print()

print(np.prod(a))
print(a.prod())

In [ ]:
# Even though the data is in a two-dimensional array,
# these functions examine all the data.
# Imagine trying to do this with Python lists!
print(a)
print()

print(f"`a` max is {np.max(a)}")
print(f"`a` min is {np.min(a)}")

In [ ]:
# Alternatively:
print(a)
print()

print(f"`a` max is {a.max()}")
print(f"`a` min is {a.min()}")

In [ ]:
# we can target a specific "axis", however, to get row-wise and column-wise results.
# `axis = 0` is for processing across columns
# `axis = 1` is for processing across rows
print(a)
print()

print("Max across columns:")
print(np.max(a, axis=0))
print()

print("Max across rows:")
print(a.max(axis=1))

In [ ]:
# We can also find the indices of the smallest and largest values.
print(a)
print()

print("Index of smallest value:")
print(np.argmin(a))
print()

# or, by row
print("Index of smallest value, by row:")
print(a.argmin(axis=1))
print()

# or, by column
print("Index of smallest value, by column:")
print(a.argmin(axis=0))

In [ ]:
# Some stats
print(a)
print()

print("Average:")
print(a.mean())
print()

# Average value by column
print("Average value by column:")
print(a.mean(axis=0))

In [ ]:
print(a)
print()

print("standard deviation:", a.std())
print()

print("standard deviation by column:\n  ", a.std(axis=0))
print()

print("standard deviation (sample) by column:\n  ", a.std(axis=0, ddof=1))

In [ ]:
print(a)
print()

print("variance:", a.var())
print()

print("varianceby column:\n  ", a.var(axis=0))

# Matplotlib

From the Matplotlib website, "Matplotlib is a comprehensive library for creating static, animated, and interactive visualizations in Python."

The goal of this "short" introduction is to introduce you to the basic ideas and concepts that Matplotlib uses, so that you can continue in your own studies and have a fundamental understanding of how the parts are structured.


**Documentation:** https://matplotlib.org/contents.html

**Tutorials:** https://matplotlib.org/tutorials/index.html

The first command that you will see is `%matplotlib notebook`.  This is **not** a Python command, but a Jupyter Notebook command.  It tells Jupyter to display the Matplotlib resuls (the chart or graph) in the notebook, rather than opening a new window.

Just as it is common to import `numpy` as `np`, it is common to import `matplotlib.pylab` as `plt`.

In [ ]:
%matplotlib notebook
import matplotlib.pylab as plt
import numpy as np

In [ ]:
x = np.linspace(0, 10)
plt.plot(x, np.sin(x), '-o', linewidth=2)
plt.plot(x, np.cos(x))

# Not needed in Notebooks, but good to include in case this code
# is used in a stand-alone script.
plt.show()

Let's break this down.

* `x` is just an array of values, from 0 to 10.
* plt.plot() is used twice, once for each line to be added to the graph.
    * The first line (blue) is given the "x" values and the "y" values separately.  "x" is the `x` numpy array.  "y" is `np.sin(x)`.
    * Additional arguments set the drawing style (a line with dots) and the width of the line.
    * The second line (orange) is, obviously, drawing the cosine function.
    
Let's see a few more examples, to help us to build our intuition about what's going on.

In [ ]:
plt.plot([1,3,2,4])
plt.show()

**Wait!?!  Why did that line get added to the previous graph?!?**

This is the first example of Matplotlib *hiding* information from you.

Basically, there is a hidden variable representing the current graph.  Every time that you do something to change the graph, it changes the graph that is pointed to internally.  You can actually save a reference to a graph in a variable, but that is only when you need to create complex layouts.

For our purposes at the moment, we will use the command `plt.figure()` to tell matplotlib that we want to start fresh on a new graph.

In [ ]:
plt.figure() # Start a new drawing
plt.plot([1,3,2,4])
plt.show()

**Wait!**

Notice that, in the line `plt.plot([1,3,2,4])`, we only gave the equivalent of `y` values!  Since we did not provide our own `x` values, Matplotlib will supply the numbers `0,1,2,3,...` as needed for the corresponding `x` values, as you can see on the graph above.

What happens if we try to add another line, this time with a **different** number of points to plot?

In [ ]:
plt.plot([6,7,8,9,0])
plt.show()

As you can see, Matplotlib **doesn't care** that this new line has a different number of data points.  It just plots what you tell it to!

In [ ]:
# clears the figure
plt.clf()
# but I don't like this approach

Let's add some information to the graph.  First, we'll start with a simple graph.

In [ ]:
# Create a new figure
plt.figure()
plt.plot([1,3,2,4])
plt.show()

As you can see, the `x` axis values are filled in with increments of `.5`.  But that doesn't make sense with our data, so let's **change the `x` axis tick marks** using the `plt.xticks()` function.

**Documentation:** https://matplotlib.org/api/_as_gen/matplotlib.pyplot.xticks.html

In [ ]:
# https://matplotlib.org/api/_as_gen/matplotlib.pyplot.xticks.html
plt.figure()
plt.plot([1,3,2,4])
plt.xticks([1,2,3,4])
plt.show()

This **works**, but the labels don't really line up with the data points.  Let's see if we can figure out why.

In [ ]:
plt.figure()
plt.plot([1,3,2,4])
plt.xticks([1,3])
plt.show()

If we just ask for x-ticks `[1,3]`, then evidently this is referring to the **values** of `1` and `3`.

Think of this as being able to specify **where** the x-ticks should appear along the x axis.

Of course, then you might think that we should be able to change **what** the x-tick label is (so that it is not just a number).  We can do this by adding a 2nd argument to the `plt.xticks()` function call.

In [ ]:
plt.figure()
plt.plot([1,3,2,4])
plt.xticks([1,2,3,4], ['John', 'Paul', 'Ringo', 'George'])
plt.show()

Of course, the data still isn't aligned with the labels.

Let's shift the x-axis labels (tick marks) by using values 0-3 rather than the original 1-4.

In [ ]:
plt.figure()
plt.plot([1,3,2,4])
plt.xticks([0,1,2,3], ['John', 'Paul', 'Ringo', 'George'])
plt.show()

And, as usual in Python, we can streamline the x-tick value list with a `range()` (or comprehension).

In [ ]:
plt.figure()
plt.plot([1,3,2,4])
plt.xticks(range(4), ['John', 'Paul', 'Ringo', 'George'])
plt.show()

While we're at it, let's make the y axis tick marks say something useful, too.

In [ ]:
plt.figure()
plt.plot([1,3,2,4])
plt.xticks(range(4), ['John', 'Paul', 'Ringo', 'George'])
plt.yticks([0,3], ['not hungry', 'hungry'])
plt.show()

Customizing the line is easy, simply by passing in additional arguments to the `plt.plot()` function.  In the following example, the line is red, dotted, and the data points are a down-pointing arrow.

**Documentation:** https://matplotlib.org/api/_as_gen/matplotlib.pyplot.plot.html

In [ ]:
# https://matplotlib.org/api/_as_gen/matplotlib.pyplot.plot.html
plt.figure()
plt.plot([1,3,2,4], 'r:v')
plt.xticks(range(4), ['John', 'Paul', 'Ringo', 'George'])
plt.show()

It should be noted that Matplotlib **doesn't actually care** about where you want to place the tick marks or what label you assign to them.  The following example exists only to show you that the label positions can be highly customized, irrespective of the data being displayed along that axis.

In [ ]:
# https://matplotlib.org/api/_as_gen/matplotlib.pyplot.plot.html
plt.figure()
plt.plot([1,3,2,4], 'r:v')
plt.xticks([.25, .5, 3, 3.5], ['John', 'Paul', 'Ringo', 'George'])
plt.show()

Let's add more information to the graph.

* Title: using `plt.title()`
* Legend:
    * First, add a `label` arguemnt to each `plt.plot()` function call.
    * Second, call `plt.legend()`
* Label the x-axis: `plt.xlabel()`
* Label the y-axis: `plt.ylabel()`

For more information, see:

**Parts of a figure**

https://matplotlib.org/tutorials/introductory/usage.html#sphx-glr-tutorials-introductory-usage-py

**Understanding plot() options**

https://matplotlib.org/1.5.3/api/pyplot_api.html#matplotlib.pyplot.plot

In [ ]:
plt.figure()
x = np.linspace(0, 10)
plt.plot(x, np.sin(x), '-o', linewidth=2, label="Sin")
plt.plot(x, np.cos(x), label="Cos")
tick = np.arange(4) * np.pi
plt.xticks(tick, ['0', 'π', '2π', '3π'])
plt.legend()
plt.title("Sad Trigonometry")
plt.xlabel("Radians")
plt.ylabel("Value")
plt.show()

In [ ]:
t = np.linspace(0,2*np.pi,50)
x = np.sin(t)
y = np.cos(t)

plt.figure()
plt.plot(t,x)
plt.title("Sin")

plt.figure()
plt.plot(t, y)
plt.title("Cos")
plt.show()

plt.figure()
plt.plot(y,x)
plt.title("Circle")
plt.show()


Again, another weird issue!  The circle at in the last value should be... well... **round!!!***

Of course, the problem is that neither Python nor Matplotlib knows that we wanted the shape to be round!  It has a default size, and tries to make the data fit into that default size window.  (Usually the default size is 4x5.)

We can change the size, but we need to get a **reference** to the figure itself, which we can do by calling the `plt.gcf()` function (**g**et **c**urrent **f**igure).  Then, just call the figure's `.set_size_inches()` method.

In [ ]:
fig = plt.gcf()

fig.set_size_inches(4,4)

As you can see, **not all lines go from just left to right!**

In fact, Matplotlib doesn't care at all if your line wraps back on itself.  It just draws from one (x,y) coordinate to the next.

In [ ]:
import random
random.seed(42)

plt.figure()

points = [(random.randrange(10), random.randrange(10)) for _ in range(10)]

y = [point[0] for point in points]
x = [point[1] for point in points]

plt.plot(y,x)
plt.title("Random Points.")

plt.show()

print()
print(points)


## Challenge

**Create a line graph of the popularity of a person's name over time.**

In [ ]:
searchName = "Corey"


def getNamePopularityByYear(name):
    data = {}
    for state, sex, year, recordName, count in babyList:
        data.setdefault(year,0)
        if name == recordName:
            data[year] += count
    return [(year, data[year]) for year in range(min(data.keys()), max(data.keys())+1)]

data = getNamePopularityByYear(searchName)

plt.figure()

x = [year for (year, count) in data]
y = [count for (year, count) in data]

plt.plot(x, y, label=searchName)

plt.legend()
plt.title("Name Popularity")

plt.show()

**Now, graph the data for two names.**

In [ ]:


def getNamePopularityByYear(name):
    data = {}
    for state, sex, year, recordName, count in babyList:
        data.setdefault(year,0)
        if name == recordName:
            data[year] += count
    return [(year, data[year]) for year in range(min(data.keys()), max(data.keys())+1)]


plt.figure()

nameList = ["Corey", "Kelsey", "Emily", "James"]

for name in nameList:
    data = getNamePopularityByYear(name)


    x = [year for (year, count) in data]
    y = [count for (year, count) in data]

    plt.plot(x, y, label=name)

plt.legend()
plt.title("Name Popularity")

plt.show()

## Saving The Graph As An Image

You can save the graph to a file using the [`plt.savefig()` function](https://matplotlib.org/api/pyplot_api.html#matplotlib.pyplot.savefig).  It is quite straightforward to use, although can be customized (dpi, transparency, size) via the function arguments.

In [ ]:
searchName = "Corey"


def getNamePopularityByYear(name):
    data = {}
    for state, sex, year, recordName, count in babyList:
        data.setdefault(year,0)
        if name == recordName:
            data[year] += count
    return [(year, data[year]) for year in range(min(data.keys()), max(data.keys())+1)]

data = getNamePopularityByYear(searchName)

plt.figure()

x = [year for (year, count) in data]
y = [count for (year, count) in data]

plt.plot(x, y, label=searchName)

plt.legend()
plt.title("Name Popularity")

plt.grid()

plt.show()

plt.savefig("namePopularity.png")

## More Graph Types

Matplotlib supports [a lot](https://matplotlib.org/3.1.0/tutorials/introductory/sample_plots.html) of graph types!

They all follow the same general philosophy: render the data points that you provide.

As an example, let's consider a simple bar chart:

In [ ]:
plt.figure()

values = [1,3,5,6,3]

plt.bar(range(len(values)), values)

plt.show()

As you can see, we had to provide **2** sets of values to the function:
1. `range(len(values))` serves as the x coordinates of where to draw the bars.
2. `values` are the height of each bar.

We can, of course, modify the horizontal placement of the bars, and we will also modify the width of the bars in order to make their placement more obvious:

In [ ]:
plt.figure()

values = [1,3,5,6,3]

plt.bar([.5, 1, 1.5, 4, 4.5], values, width=.25)

plt.show()

And what if you wanted to graph more than one data set?  Well, that is up you to you!  You must decide where to put each set of data in relation to one another (which will probably involve some math).

Consider the following two examples:

In [ ]:
import string

plt.figure()

widgetAData = [1,3,5,6,3]
widgetBData = [7,1,3,4,2]

plt.bar([i for i in range(len(widgetAData))], widgetAData, width=.25, label="Widget 1")
plt.bar(np.linspace(.25, 4.25, 5), widgetBData, width=.25, label="Widget 2")

xtickOffset = [i + .125 for i in range(len(widgetAData))]
#xtickLabel = [chr(ch) for ch in range(ord('A'), ord('A') + len(widgetAData))]
xtickLabel = [*string.ascii_uppercase[:len(widgetAData)]]
plt.xticks(xtickOffset, xtickLabel)

plt.legend()
plt.title("Widgets")

plt.show()

In [ ]:
plt.figure()

widgetAData = [1,3,5,6,3]
widgetBData = [7,1,3,4,2]

barPosition = np.linspace(0, 2, 5)
plt.bar(barPosition, widgetAData, width=.25, label="Widget 1")
plt.bar(barPosition + 3, widgetBData, width=.25, label="Widget 2")

xtickOffset = [i/2 for i in range(len(widgetAData))] + [i/2 + 3 for i in range(len(widgetAData))]
xtickLabel = [chr(ch) for ch in range(ord('A'), ord('A') + len(widgetAData))] * 2
plt.xticks(xtickOffset, xtickLabel)

plt.legend()
plt.title("Widgets")

plt.show()

In both of the preceding cases, the data for the two "Widgets" stayed the same, but we were able to get different **groupings** by controlling the `x axis values` that we provided to the `plt.bar()` function.

And, of course, you can combine different types of graphs:

In [ ]:
plt.figure()

widgetAData = [1,3,5,6,3]
widgetBData = [7,1,3,4,2]

plt.bar(range(len(widgetAData)), widgetAData, width=.25, label="Widget A")
plt.plot(range(len(widgetBData)), widgetBData, 'ko-.', label="Widget B")

xtickOffset = [i + .125 for i in range(len(widgetAData))]
xtickLabel = [chr(ch) for ch in range(ord('A'), ord('A') + len(widgetAData))]
plt.xticks(xtickOffset, xtickLabel)

plt.legend()
plt.title("Widgets")

plt.show()

## Challenge:

**Draw a pie chart of the overall popularity for the requested names.**

https://matplotlib.org/3.1.0/api/_as_gen/matplotlib.pyplot.pie.html#matplotlib.pyplot.pie

In [ ]:
names = ["Matthew", "Mark", "Luke", "John"]
#names = ["John", "Paul", "Ringo", "George"]
#names = ["Peter", "Andrew", "James", "John", "Philip", "Thaddeus", "Bartholemew", "Thomas", "Matthew", "Simon", "Judas"]

def getNamePopularity(names):
    data = {name:0 for name in names}
    for state, sex, year, name, count in babyList:
        if name in data:
            data[name] += count
    return data

plt.figure()

data = [*getNamePopularity(names).items()]

plt.pie([count for (name, count) in data], labels=[name for (name, count) in data])

plt.show()


Demo of a scatter plot:

https://matplotlib.org/3.1.0/api/_as_gen/matplotlib.pyplot.scatter.html

In [ ]:
import random

random.seed(1000)

x = [random.random() for _ in range(10)]
y = [random.random() for _ in range(10)]

plt.figure()
plt.scatter(x, y)
plt.show()

You can change the size of each marker by providing a either a single number (all markers will have the same size) or a list of sizes, which will be applied to each mark in the order specified in the x,y coordinates.

In [ ]:
import random

random.seed(1000)

x = [random.random() for _ in range(10)]
y = [random.random() for _ in range(10)]
sizes = np.linspace(2,100,10)

plt.figure()
plt.scatter(x, y, s=sizes)
plt.show()

## Stupid Matplotlib Tricks

**XKCD Style**

You can make your charts have the styling of the great [XKCD Comics](https://xkcd.com/).

Because I don't want to influence other charts, I will use the `with...:` syntax.

In [ ]:


def getNamePopularityByYear(name):
    data = {}
    for state, sex, year, recordName, count in babyList:
        data.setdefault(year,0)
        if name == recordName:
            data[year] += count
    return [(year, data[year]) for year in range(min(data.keys()), max(data.keys())+1)]


# Only use the xkcd style on this one graph:
with plt.xkcd():
    plt.figure()

    nameList = ["Corey", "Kelsey", "Emily", "James"]

    for name in nameList:
        data = getNamePopularityByYear(name)


        x = [year for (year, count) in data]
        y = [count for (year, count) in data]

        plt.plot(x, y, label=name)

    plt.legend()
    plt.title("Name Popularity")

    plt.show()

See the gallery for examples covering many different features!

https://matplotlib.org/3.1.0/gallery/index.html

See "Anatomy of a figure" for a thorough explanation of the different parts that you can control inside a figure.

https://matplotlib.org/3.1.0/gallery/showcase/anatomy.html#sphx-glr-gallery-showcase-anatomy-py

## The Object-Oriented Interface

As it turns out, Matplotlib can be complicated, but that it a result of its flexibility and power.

So far, we have been using Python to manipulate Matplotlib in a very straightforward way.  We have told Matplotlib to create a new figure, to draw on that figure, and then we move to the next figure.  This is the **simple** way to use Matplotlib.  There are other, more advanced ways.

What if you want to have multiple graphs side-by-side or overlapping?  What if you want to work on multiple graphs at the same time?  For this, we need an **object-oriented** approach.

First, you need to understand the difference between the words **figure**, **axes**, and **axis**.

* **axis**: This is simple and straightforward.  The graphs that we have seen so far have contained an x-axis and a y-axis.
* **axes**: This is a bad name, in my opinion.  It is **not** the plural of "axis", but rather it referrs to the drawing area that we add our lines to.
* **figure**: This contains the axes.  There may be more than one axes, as we will see in a moment.

In the next example, notice that we save a reference to the **figure** and the **axes** by making a call to the [`plt.subplots()` function](https://matplotlib.org/3.1.0/api/_as_gen/matplotlib.pyplot.subplots.html#matplotlib.pyplot.subplots).

All plotting is done against the **`ax`** object.

Showing is done against the **`fig`** object.

In [ ]:
fig, ax = plt.subplots()

x = np.linspace(0, 10)
ax.plot(x, np.sin(x), '-o', linewidth=2)
ax.plot(x, np.cos(x))

# Not needed in Notebooks, but good to include in case this code
# is used in a stand-alone script.
fig.show()

But what is all of this about subplots?

It's easier to show you.  In the preceding example, notice that `ax` was a reference to the drawing area.

In this next example, we will ask for **2** subplots, and so `ax` will be a **numpy array** of references to the two requested drawing areas.

We can target the drawing area for each separately by addressing them as `ax[0]` and `ax[1]`.

In [ ]:
fig, ax = plt.subplots(2)

print("They type of `ax` is:", type(ax))
print("`ax` contains:")
print(ax)

x = np.linspace(0, 10)
ax[0].plot(x, np.sin(x), '-o', linewidth=2)
ax[1].plot(x, np.cos(x))

# Not needed in Notebooks, but good to include in case this code
# is used in a stand-alone script.
fig.show()

Of course, rather than having 2 rows of axes, why not have 1 row and two columns instead?

We just need to change the arguments given to `plt.subplots()`.

In [ ]:
fig, ax = plt.subplots(1,2)

print("They type of `ax` is:", type(ax))
print("`ax` contains:")
print(ax)

x = np.linspace(0, 10)
ax[0].plot(x, np.sin(x), '-o', linewidth=2)
ax[1].plot(x, np.cos(x))

# Not needed in Notebooks, but good to include in case this code
# is used in a stand-alone script.
fig.show()

We can even do more complicated layouts, just by asking for a more complicated layout from `plt.subplots()`.

In [ ]:
fig, ax = plt.subplots(2,2)

print("They type of `ax` is:", type(ax))
print("`ax` contains:")
print(ax)

x = np.linspace(0, 10)
ax[0,0].plot(x, np.sin(x), '-o', linewidth=2)
ax[1,0].plot(x, np.cos(x))
ax[0,1].pie([1,2,3], labels=["A", "B", "C"])
ax[1,1].bar(range(5), [1,3,2,4,-3])

# Not needed in Notebooks, but good to include in case this code
# is used in a stand-alone script.
fig.show()

For more information, check out the following links:

https://matplotlib.org/faq/usage_faq.html#usage

https://matplotlib.org/3.1.0/api/_as_gen/matplotlib.pyplot.subplots.html#matplotlib.pyplot.subplots (scroll down for **a lot** of really great examples!)

https://matplotlib.org/3.1.0/gallery/subplots_axes_and_figures/subplots_demo.html (good examples of combining the axis of different subplots)

https://matplotlib.org/3.2.1/tutorials/intermediate/constrainedlayout_guide.html (if you *really*, **really**, ***really*** want some complex layouts)

Examples from the complex layouts link:

In [ ]:
import matplotlib.colors as mcolors
arr = np.arange(100).reshape((10, 10))
norm = mcolors.Normalize(vmin=0., vmax=100.)
pc_kwargs = {'rasterized': True, 'cmap': 'viridis', 'norm': norm}

def example_plot(ax, fontsize=12, nodec=False):
    ax.plot([1, 2])

    ax.locator_params(nbins=3)
    if not nodec:
        ax.set_xlabel('x-label', fontsize=fontsize)
        ax.set_ylabel('y-label', fontsize=fontsize)
        ax.set_title('Title', fontsize=fontsize)
    else:
        ax.set_xticklabels('')
        ax.set_yticklabels('')



def docomplicated(suptitle=None):
    fig = plt.figure()
    gs0 = fig.add_gridspec(1, 2, figure=fig, width_ratios=[1., 2.])
    gsl = gs0[0].subgridspec(2, 1)
    gsr = gs0[1].subgridspec(2, 2)

    for gs in gsl:
        ax = fig.add_subplot(gs)
        example_plot(ax)
    axs = []
    for gs in gsr:
        ax = fig.add_subplot(gs)
        pcm = ax.pcolormesh(arr, **pc_kwargs)
        ax.set_xlabel('x-label')
        ax.set_ylabel('y-label')
        ax.set_title('title')

        axs += [ax]
    fig.colorbar(pcm, ax=axs)
    if suptitle is not None:
        fig.suptitle(suptitle)

docomplicated()

In [ ]:
fig = plt.figure()
gs = fig.add_gridspec(2, 2)

ax1 = fig.add_subplot(gs[0, 0])
ax2 = fig.add_subplot(gs[1, 0])
ax3 = fig.add_subplot(gs[:, 1])

example_plot(ax1)
example_plot(ax2)
example_plot(ax3)